In [18]:
import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model
import os
from pathlib import Path


In [2]:
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip -d fra-eng


--2024-07-13 06:13:49--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  14.6MB/s    in 0.5s    

2024-07-13 06:13:49 (14.6 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]

Archive:  fra-eng.zip
  inflating: fra-eng/_about.txt      
  inflating: fra-eng/fra.txt         


In [3]:
batch_size=64
epochs=100
latent_dim=256
num_samples=1000
data_path=r"/content/fra-eng/fra.txt"

In [4]:
input_texts=[]
target_texts=[]
input_character=set()    #all chars in input data (english) set()--> to take uniques
target_character=set()   #all chars in target data (france) set()--> to take uniques


In [5]:
with open(data_path,'r',encoding='utf-8')as f:
    lines=f.read().split('\n')


In [6]:
len(lines)


232737

In [7]:
# num_samples = 10000
for line in lines[: min(num_samples, len(lines))]:
  input_text, target_text, _ = line.split("\t")
  target_text = "\t" + target_text + "\n"

  input_texts.append(input_text)
  target_texts.append(target_text)

  for char in input_text:
    if char not in input_character:
      input_character.add(char)
  for char in target_text:
    if char not in target_character:
      target_character.add(char)

In [8]:
input_character

{' ',
 '!',
 "'",
 ',',
 '.',
 '1',
 '9',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [9]:
for i in range(15):
  print(input_texts[i],target_texts[i])


Go. 	Va !

Go. 	Marche.

Go. 	En route !

Go. 	Bouge !

Hi. 	Salut !

Hi. 	Salut.

Run! 	Cours !

Run! 	Courez !

Run! 	Prenez vos jambes à vos cous !

Run! 	File !

Run! 	Filez !

Run! 	Cours !

Run! 	Fuyez !

Run! 	Fuyons !

Run. 	Cours !



In [10]:
input_character=sorted(list(input_character))
target_character=sorted(list(target_character))

num_encoder_tokens=len(input_character) #no. of characters
num_decoder_tokens=len(target_character)

In [11]:
num_encoder_tokens,num_decoder_tokens

(56, 76)

In [12]:
max_encoder_seq_length=max([len(txt) for txt in input_texts]) #to get max length of word
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [13]:
max([len(txt) for txt in input_texts])

10

In [14]:
max([len(txt) for txt in target_texts])

33

In [15]:
input_token_index=dict([(char,i) for i,char in enumerate(input_character)])
target_token_index=dict([(char,i) for i,char in enumerate(target_character)])

In [16]:
input_token_index

{' ': 0,
 '!': 1,
 "'": 2,
 ',': 3,
 '.': 4,
 '1': 5,
 '9': 6,
 '?': 7,
 'A': 8,
 'B': 9,
 'C': 10,
 'D': 11,
 'E': 12,
 'F': 13,
 'G': 14,
 'H': 15,
 'I': 16,
 'J': 17,
 'K': 18,
 'L': 19,
 'M': 20,
 'N': 21,
 'O': 22,
 'P': 23,
 'R': 24,
 'S': 25,
 'T': 26,
 'U': 27,
 'W': 28,
 'Y': 29,
 'a': 30,
 'b': 31,
 'c': 32,
 'd': 33,
 'e': 34,
 'f': 35,
 'g': 36,
 'h': 37,
 'i': 38,
 'j': 39,
 'k': 40,
 'l': 41,
 'm': 42,
 'n': 43,
 'o': 44,
 'p': 45,
 'q': 46,
 'r': 47,
 's': 48,
 't': 49,
 'u': 50,
 'v': 51,
 'w': 52,
 'x': 53,
 'y': 54,
 'z': 55}

In [17]:
#encoder_input(english text)
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
#decoder_input(intial hidden state from encoder)
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
#decoder_output(french text)
decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [24]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.0  #get index of char and put 0 in it
  encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0 #padding

  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1.0
    if t > 0:
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
  decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
  decoder_target_data[i, t:, target_token_index[" "]]=1.0

In [25]:
#encoder layer
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)

encoder_output , state_h , state_c =encoder(encoder_inputs)
encoder_states=[state_h,state_c]

In [26]:
#encoder layer
decoder_inputs = Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)

decoder_output,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_output)

In [28]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [29]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])


In [31]:
model.fit([encoder_input_data,decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
13/13 [==============================] - 5s 430ms/step - loss: 2.2383 - accuracy: 0.5987 - val_loss: 1.8809 - val_accuracy: 0.5658
Epoch 2/100
13/13 [==============================] - 4s 353ms/step - loss: 1.7592 - accuracy: 0.5994 - val_loss: 1.7932 - val_accuracy: 0.5658
Epoch 3/100
13/13 [==============================] - 5s 375ms/step - loss: 1.6422 - accuracy: 0.6072 - val_loss: 1.7152 - val_accuracy: 0.6002
Epoch 4/100
13/13 [==============================] - 5s 349ms/step - loss: 1.5965 - accuracy: 0.6272 - val_loss: 1.6445 - val_accuracy: 0.5994
Epoch 5/100
13/13 [==============================] - 3s 269ms/step - loss: 1.5700 - accuracy: 0.6303 - val_loss: 1.6220 - val_accuracy: 0.6003
Epoch 6/100
13/13 [==============================] - 3s 202ms/step - loss: 1.5543 - accuracy: 0.6270 - val_loss: 1.6582 - val_accuracy: 0.5976
Epoch 7/100
13/13 [==============================] - 3s 230ms/step - loss: 1.4918 - accuracy: 0.6309 - val_loss: 1.8872 - val_accuracy: 0.5903

In [32]:
model.save_weights('seq2seq_translatio_model.h5')

#Inference Sampling

In [33]:
# here is the drill :
# 1) encode input and retrive intial decoder state
# 2) run one step of decoder with this intial state
# and a "start sequence" token as target.
# output will be the next target token
# 3) repeat with the current target token and current states

#Define sampling models
encoder_model=Model(encoder_inputs,encoder_states)

In [34]:
decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]
decoder_outputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
decoder_states=[state_h,state_c]
decoder_outputs=decoder_dense(decoder_outputs)

decoder_model=Model([decoder_inputs]+decoder_states_inputs,
 [decoder_outputs]+decoder_states)

In [35]:
# Reverse-lookup token index to decode sequences back to something readable (index-->char)
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [36]:
def decode_sequence(input_seq):
  #encode the input as state vectors.
  states_value=encoder_model.predict(input_seq)

  #generate empty target sequence of length 1.
  target_seq=np.zeros((1,1,num_decoder_tokens))
  #populate the first charachter of target sequence with start character
  target_seq[0,0,target_token_index['\t']] =1.

  #sampling loop for a batch of sequences
  #(to simplify,here we assume a batch of size 1)
  stop_condition=False
  decoded_sentence=''
  while not stop_condition:
    output_tokens,h,c=decoder_model.predict(
        [target_seq]+states_value)

    #sample a token
    sampled_token_index=np.argmax(output_tokens[0,-1,:])
    sampled_char=reverse_target_char_index[sampled_token_index]
    decoded_sentence+=sampled_char

    #exit condition: either hit max length or find
    if(sampled_char=='\n' or
       len(decoded_sentence)>max_decoder_seq_length):
      stop_condition=True

    #update the target sequence(of length 1)
    target_seq=np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,sampled_token_index]=1.

    #update states
    states_value=[h,c]

  return decoded_sentence


In [38]:
for seq_index in range(50):
  #take one sequence (part of the training set)
  # for trying out decoding
  input_seq=encoder_input_data[seq_index:seq_index+1]
  decoded_sentence=decode_sequence(input_seq)
  print('-')
  print('Input sentence:',input_texts[seq_index])
  print('Decoded sentence:',decoded_sentence)




1/1 [==============================] - 0s 24ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Go.
Decoded sentence: Va !

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Hi.
Decoded sentence: Sontren !

1/1 [==============================] - 0s 27ms/step
-
Input sentence: Hi.
Decoded sentence: Sontren !

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Run!
Decoded sentence: Fuis-j                            
1/1 [==============================] - 0s 43ms/step
-
Input sentence: Run!
Decoded sentence: Fuis-j                            
1/1 [==============================] - 0s 23ms/step
-
Input sentence: Run!
Decoded sentence: Fuis-j                            
1/1 [=============